![title](figure_1_burr_settles.png)    

Fig 1 Credit: Burr Settles. 2009. “Active Learning Literature Survey.” Computer Sciences Technical Report 1648. University of Wisconsin–Madison.


This Notebook demonstrates the process of Active Learning (AL) using features from the Stanford movies review dataset (http://ai.stanford.edu/~amaas/data/sentiment/) following the CRISP-DM process (https://paginas.fe.up.pt/~ec/files_0405/slides/02%20CRISP.pdf).  


**Statement of Business Objective**

The business would like to gain more value from unstructured text data while optimizing use of labor resources.

**Statement of Machine Learning Objective**

The machine learning system should enable the business to indepententy generate concepts and perform classification on their own data.  The system will use AL to deliver increased value from unstructured data.

AL accomplishes this through an iterative process that trains on the labeled features and then predicts on the unlabeled samples' features.  These predictions can then be evaluated in a manner to suggest the next desirable review for the expert to label.  

For example, ordering based on distance from the classification boundary allows prioritization of the sample that may be questionable.  Once labeled, the iterative process repeats, the labeled features are used to train the model, and predictions are evaluated to provide the next sample(s) for labeling.

**Statement of Success Criteria**  

AL is a process that relies on an expert's feedback to label samples and drive the machine learning process.  The benefit of Active Learning with Rationales is to learn a model of the concept (in this case sentiment) in a manner that further minimizes the need for the expert's time (resources).  Thus this effort will show the advantage of using rationales to reduce the need for business resources.

**Data Understanding** 

In the case of sentiment (as the concept), the positive and negative labels simply a boolean value for each sample.  Here is a positive sentiment sample from the movie review dataset:
    

**[train/pos/1999_9.txt]**   
*Oliver! the musical is a favorite of mine. The music, the characters, the story. It all just seems perfect. In this rendition of the timeless classic novel turned stage musical, director Carol Reed brings the Broadway hit to life on the movie screen.<br /><br />The transition from musical to movie musical is not an easy one. You have to have the right voices, the right set, the right script, and the right play. All signs point to yes for this play. It almost appears that it was written for the screen!<br /><br />Our story takes place in jolly old England where a boy named Oliver manages to work his way out of the orphanage. He winds his way through the country to London where he meets up with a group of juvenile delinquents, headed by Dodger, the smart talking, quick handed pick-pocket. The leader of this gang is named Fagin, an older fellow who sells all the stolen goods.<br /><br />But all is not well in London town when Bill Sykes played by Oliver Reed and his loving girlfriend Nancy get tangled up with Oliver, Fagin and his young troops, and the law. What ensues is a marvelous tale of love, affection, and great musical numbers.<br /><br />Whether or not you like musicals or not, one listen to these tunes and you will be humming them all day long. Oliver! is a triumph on and off the stage and is a timeless work of art.*

**Data Preparation**

The movie review data dataset does not have rationales.  Amazon's Mechanical Turk provides a way to gather the rationales for both the positive and negative reviews.  The first acceptable rationale for each review is selected for a balanced number of postive and negative samples.  A pretrained word2vec model is used to generate word level features embeddings.  Then these results are saved for the next stage since they are used many times (as seen later for grid search and shuffle processing).

Here is the source code for filtering the input data:

In [4]:
!cat process_step_1_read_split_save.py


import pandas as pd
import random
import numpy as np
import pickle

# load data
pos_df = pd.read_csv('../../../Downloads/Batch_3494319_batch_results.csv')
neg_df = pd.read_csv('../../../Downloads/Batch_3494476_batch_results.csv')

# only get the accepted records from the mechanical turk results
pos_df_approved = pos_df[pos_df['AssignmentStatus']=='Approved']
neg_df_approved = neg_df[neg_df['AssignmentStatus']=='Approved']

# get a balanced set of highlighted postive and negative reviews
pos_df_approved = pos_df_approved.drop_duplicates(subset='Input.text', keep='first')
neg_df_approved = neg_df_approved.drop_duplicates(subset='Input.text', keep='first')
balanced_count = min(len(pos_df_approved), len(neg_df_approved))

# buid the dataset to contain the 
labeled_pos = random.sample(pos_df_approved['Answer.highlights'].tolist(), balanced_count)
labeled_neg = random.sample(neg_df_approved['Answer.highlights'].tolist(), balanced_count)

# here, negative is mapped to zere and positive to on

Here is the source code for finding and saving the features:

In [6]:
!cat process_step_2_assign_features.py

import pickle
import numpy as np
import pymagnitude

# load input data
X = pickle.load( open('X.pkl', 'rb') )

# load the pretrained word2vec model for feature assignment
pretrained_magnitude = r'../../../Downloads/pretrained/glove.6B.300d.magnitude'
vectors = pymagnitude.Magnitude(pretrained_magnitude)

# setup speciality cleaning
def get_document_features(data_in):
    """Used to clean 80k Mechanical Turk responses.

    Params:
        data_in --  text segment to process
    Returns:
        features for input text and features
    """
    data_in = data_in.replace('<span class=\"active_text\">', '').replace('</span>', '')
    body = data_in.split(r'\n                                    ')[1].replace('\n', '')
    avg_vec = np.mean(vectors.query(body.split(' ')), axis=(0))
    
    if highlights == True:
        high_text = data_in.split(r'\n                                    ')[0].replace('\n', '')
        high_avg_vec = np.mean(vectors.query(high_text.split(' ')), axis=(0))
     

**Modeling**

We want to first find 

In [8]:
!cat process_step_3_train_score_auc.py

# open the upstream data
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score
import numpy as np

def find_nearest(array, value):
    """Finds the prediction closest to the boundary"""
    array = np.asarray(array)
    return (np.abs(array - value)).argmin()

# different values of c (.1 increments)
for c in [0.5,0.2,0.8,0.4,0.6,0.7,0.3,0.9,0.1,0.0,1.0]:
    # import the feature data and labels
    Y = pickle.load(  open('Y.pkl', "rb" ) )
    X_reg_features = pickle.load(  open('X_reg_features.pkl', "rb" ) )
    X_high_features = pickle.load(  open('X_high_features.pkl', "rb" ) )

    # apply c to show how different proportions impact AL performance
    for idx,f in enumerate(X_high_features):
        X_high_features[idx] =  (X_high_features[idx] * c) + (1-c) * f

    # reshuffle 20 times and take the average
    # for episode in range(1,20):
    ep

**Evaluation**

**Deployment**

**Part 2 Technical Demonstration**  

In [3]:
# get information about the features, these are precomputed and cached locally
!ls /Users/username/Downloads/features/ | wc -l

  100000


Okay, we have a total of 100,000 movie reviews.

In [ ]:
# get information about a review's features
import numpy as np
dat = np.fromfile('/Users/username/Downloads/features/train_pos_1999_9.npy',dtype=np.float32) 
print(len(dat))
print(str(dat))
    

We see that the 300 features listed above are float values.  These particular features represent the positive movie review for *Oliver!*. 

Lets cover how these 300 features are generated.  Each word in the document is used to find its **word2vec** feature, and then these features are averaged to produce the above document-level feature.  Services like **pymagnitude** are used to create the 300 floats for each word using the query method.  Then, for each document, these are averaged. 

With a technical understanding of how the document's feature vector is made, let's move forward to see how the iterative Active Learning Service could be implemented using Scikit-learn.

So what do you think?  Positive or negative?  Is it apparent why the model may have a harder time classifying this review based on the author's opinion of Justin Timberlake's performance?

The Active Learning process repeats by retraining on a slightly larger labeled set of training data.  The updated model predicts on the somewhat smaller collection of unlabeled data so that the service can return the next best unlabeled review...